In [2]:
from sklearn.datasets import fetch_openml
db = fetch_openml('Fashion-MNIST', version=1)

/home/noah/anaconda3/envs/torch_38/lib/python3.8/site-packages/sklearn/datasets/_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [3]:
from torch.utils.data import TensorDataset
import torch
import pandas as pd

class DLDataset(TensorDataset):
    def __init__(self, x_data, y_data):
        super().__init__()
        x_data = x_data.values if isinstance(x_data, pd.DataFrame) else x_data
        y_data = y_data.values if isinstance(y_data, pd.DataFrame) else y_data
        
        self.feature=torch.FloatTensor(x_data)
        self.target=torch.LongTensor(y_data)
    def __len__(self):
        return self.target.shape[0]
    def __getitem__(self, index):
        return self.feature[index], self.target[index]

In [4]:
# 과제 목표 : 다중 분류 + 검증 데이터셋 사용(no_grad)
feature = db.data # 피쳐 설정
target = db.target # 타겟 설정 

In [5]:
from sklearn.preprocessing import LabelEncoder
target = LabelEncoder().fit_transform(target)
target=target.reshape(-1,1)
my_dataset=DLDataset(feature, target)

In [6]:
from torch.utils.data import random_split

seed=torch.Generator().manual_seed(42)
trainDS, validDS, testDS = random_split(my_dataset, [0.7,0.1,0.2], generator=seed)

In [7]:
from torch.utils.data import DataLoader 
batchsize=5
trainDF=DataLoader(trainDS, batch_size=batchsize, shuffle=True)
validDF=DataLoader(validDS, batch_size=batchsize, shuffle=True)
testDF=DataLoader(testDS, batch_size=batchsize, shuffle=True)

In [9]:
import torch.nn as nn

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
EPOCHS = 10
LOSS_FN=nn.CrossEntropyLoss().to(DEVICE)

In [13]:
import torch.nn.functional as F
import torch.optim as optim


class Model(nn.Module): # 오늘 배운 거! 
    def __init__(self):
        super().__init__() # nn.Module 상속
        self.fc1 = nn.Linear(784, 256) # 28 X 28 = 784 이므로, 들어가는 피쳐의 값은 784개임
        self.fc2 = nn.Linear(256, 128) # 은닉층1
        self.fc3 = nn.Linear(128, 64) # 은닉층2 
        self.fc4 = nn.Linear(64, 10) # 10개의 클래스로 분류해야 하므로, 10개의 피쳐가 나오도록 설정
        
    def forward(self, x):  # 오늘 배운 거2!
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x
model = Model() # 모델 객체 생성
# 이제 옵티마이저와 손실함수를 설정해야한다! 

In [14]:
optimizer=optim.Adam(model.parameters(), lr=0.001)
import torchmetrics.functional as metrics

def training():
    loss_list=[]
    model.train()
    for cnt, (feature, target) in enumerate(trainDF):
        feature, target = feature.to(DEVICE), target.to(DEVICE)
        pre_target = model(feature.float())
        loss=LOSS_FN(pre_target, target.squeeze())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
        #print(metrics.accuracy( pre_target, target.squeeze(), task="multiclass", num_classes=3 ))
    return sum(loss_list)/len(loss_list)


In [15]:
training()

0.6443451218578528

In [16]:
def testing():
    loss_list=[]
    with torch.no_grad():
        model.eval()
        val_loss=[]
        for cnt, (feature, target) in enumerate(validDF):
            feature, target = feature.to(DEVICE), target.to(DEVICE)
            pre_target = model(feature.float())

            loss=LOSS_FN(pre_target, target.squeeze())
            #print(metrics.accuracy( pre_target, target.squeeze(), task="multiclass", num_classes=3 ))
            loss_list.append(loss.item())
    return sum(loss_list)/len(loss_list)

In [17]:
testing()

0.5784177430243082

In [35]:
#metrics.accuracy( pre_target, target.squeeze(), task="multiclass", num_classes=3\
def accuracy():
    with torch.no_grad():
            model.eval()
            alist=[]
            for cnt, (feature, target) in enumerate(testDF):
                feature, target = feature.to(DEVICE), target.to(DEVICE)
                pre_target = model(feature.float())
                alist.append(metrics.accuracy( pre_target, target.squeeze(), task="multiclass", num_classes=10))
    return sum(alist)/len(alist)

In [36]:
accuracy()

tensor(0.8451)

In [37]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=10, verbose=True)


df_list=[]
for eps in range(EPOCHS): # 50

    train_loss=training()
    valid_loss=testing()
    accuracy1 = accuracy()
    print(f'{eps}번째 반복 ==> train_loss : {train_loss:.2f}, valid_loss : {valid_loss:.2f}, 정확도 : {accuracy1:.2f}')
    df_list.append([eps, train_loss, valid_loss, accuracy1])
    scheduler.step(valid_loss)
    if scheduler.num_bad_epochs >= scheduler.patience:
        print(f"Early stopping at epoch {eps}")
        break

/home/noah/anaconda3/envs/torch_38/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


0번째 반복 ==> train_loss : 0.45, valid_loss : 0.44, 정확도 : 0.86
1번째 반복 ==> train_loss : 0.44, valid_loss : 0.43, 정확도 : 0.85
2번째 반복 ==> train_loss : 0.45, valid_loss : 0.45, 정확도 : 0.85
3번째 반복 ==> train_loss : 0.43, valid_loss : 0.44, 정확도 : 0.85
4번째 반복 ==> train_loss : 0.44, valid_loss : 0.52, 정확도 : 0.86
5번째 반복 ==> train_loss : 0.42, valid_loss : 0.54, 정확도 : 0.84
6번째 반복 ==> train_loss : 0.42, valid_loss : 0.51, 정확도 : 0.85
7번째 반복 ==> train_loss : 0.43, valid_loss : 0.55, 정확도 : 0.84
8번째 반복 ==> train_loss : 0.43, valid_loss : 0.61, 정확도 : 0.83
9번째 반복 ==> train_loss : 0.42, valid_loss : 0.65, 정확도 : 0.84
10번째 반복 ==> train_loss : 0.41, valid_loss : 0.57, 정확도 : 0.84
11번째 반복 ==> train_loss : 0.42, valid_loss : 0.55, 정확도 : 0.85
Early stopping at epoch 11
